In [30]:
import requests 
from postgres import Postgres
import dotenv
import os
import json as js

dotenv.load_dotenv('techtalk_SEC.secrets')
dotenv.load_dotenv('techtalk_SEC.env')

SEC_headers= {
    "User-Agent":"PWTTechTalkBot/1.0 Port Wallis Technologies contact - rwolfe@portwallistechnologies.com",
    "Accept":"application/json"
} 
r= requests.Request(method='GET',url=f'{os.getenv("WEB_URL")}/files/company_tickers.json',headers=SEC_headers)
s = requests.Session()
req= s.send( r.prepare())

with open(file='cik_tickers.json',mode='w') as file:
    the_json = req.json()
    
db = Postgres()
    
# always clear out the table first

db.run(f'truncate table "{os.getenv("PG_SCHEMA")}".company_ticker')

# bad for us to just toss away this but ...

a = [ db.run(  f"""insert  into "{os.getenv("PG_SCHEMA")}".company_ticker(
                cik_str,
                ticker,
                title) 
               values (
                   '{the_json[a]["cik_str"]}',
                   '{the_json[a]["ticker"]}',
                   '{the_json[a]["title"].replace("'","''")}') """ ) 
       for a in the_json]

company_cik :str = db.one(f"""select cik_str from "{os.getenv("PG_SCHEMA")}".company_ticker where ticker = 'AES' """)

req = requests.get(url=f"""{os.getenv("DATA_URL")}/submissions/CIK{company_cik.rjust(10,'0')}.json""", headers=SEC_headers)

with open(file='thedump.json',mode='w') as file:
    js.dump(req.json(),file)

submissions:dict = req.json()

recent_filings:dict = submissions["filings"]["recent"]


print( {k: recent_filings[k][0] for k in recent_filings.keys()})





{'accessionNumber': '0000874761-23-000073', 'filingDate': '2023-08-03', 'reportDate': '2023-08-03', 'acceptanceDateTime': '2023-08-03T17:22:04.000Z', 'act': '34', 'form': '8-K', 'fileNumber': '001-12291', 'filmNumber': '231141240', 'items': '2.02,7.01,9.01', 'size': 764538, 'isXBRL': 1, 'isInlineXBRL': 1, 'primaryDocument': 'aes-20230803.htm', 'primaryDocDescription': '8-K'}
